In [ ]:
from scipy.io import loadmat
from operator import index
import pandas as pd



from mne import create_info,EpochsArray,find_events
from mne import Epochs, pick_types, find_events, events_from_annotations,Annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, RawArray



In [ ]:
train = loadmat('stroke/p1_post_training.mat')
test = loadmat('stroke/P1_post_test.mat')
train=train
test=test

In [ ]:
def reannotate(x):
    if x ==-1:
        return 2
    else:
        return x


df_eeg_train = pd.DataFrame(train['y'])
df_trigger = pd.DataFrame(train['trig'])
eeg_data_train=df_eeg_train.merge(df_trigger,left_index=True,right_index=True)
eeg_data_train['0_y']=eeg_data_train['0_y'].apply(lambda x:reannotate(x))

df_eeg_test = pd.DataFrame(test['y'])
df_trigger_test = pd.DataFrame(test['trig'])
eeg_data_test=df_eeg_test.merge(df_trigger_test,left_index=True,right_index=True)
eeg_data_test['0_y']=eeg_data_test['0_y'].apply(lambda x:reannotate(x))

به خط RawArray توجه کن



In [ ]:

#n_channels = 16
sampling_freq = 256  # in Hertz
#info = create_info(n_channels, sfreq=sampling_freq)
ch_names = ['FC3','FCz','FC4','C5','C3','C1','Cz','C2','C4','C6','CP3','CP1','CPz','CP2','CP4','Pz','Stim']
ch_types = ['eeg']*16+['stim']
info = create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')
raw = RawArray(eeg_data_train.T, info)
raw.filter(8., 30., fir_design='firwin', skip_by_annotation='edge')
tmin, tmax = 0., 8.
event_id = dict(left=1, right=2)

events = find_events(raw, stim_channel='Stim')
mapping = {1: 'Left', 2: 'Right'}
onsets = events[:, 0] / raw.info['sfreq']
durations = np.zeros_like(onsets)  # assumes instantaneous events
descriptions = [mapping[event_id] for event_id in events[:, 2]]
annot_from_events = Annotations(onset=onsets, duration=durations,
                                    description=descriptions,
                                    orig_time=raw.info['meas_date'])
raw.set_annotations(annot_from_events)

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False)
# Read epochs (train will be done only between 2 and 3.5s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=2, tmax=6)
labels = epochs.events[:, -1] - 1